In [85]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

## Lecture des datas

In [61]:
df = pd.read_csv("all_data.csv", sep=";")
print(f"{len(df)} tickers")

1247 tickers


## Exclusions

### Tabac / Armes controversées

In [62]:
df2 = df.loc[(df['CWEAP_TIE']!=1) & (df['TOB_PRODUCER']!=1),:]
print(f"{len(df2)} tickers")

1237 tickers


Elimination de 10 entreprises

### Controverses 0/1

In [63]:
columns_controverse = ["E_CONTROVERSY", "S_CONTROVERSY", 'G_CONTROVERSY']

In [64]:
# Controverse E
for col in columns_controverse:
    print(f"NB {col} : {len(df.loc[df[col]<2,:])}")

NB E_CONTROVERSY : 28
NB S_CONTROVERSY : 149
NB G_CONTROVERSY : 35


In [65]:
df3 = df2.loc[~df[columns_controverse].lt(2).any(axis=1), :]
print(f"Elimination {len(df2)-len(df3)}")
print(f"Reste {len(df3)}")

Elimination 175
Reste 1062


### Note ESG < BB

In [66]:
print(f"NB : {len(df.loc[df["IVA_COMPANY_RATING"].isin(['B','CCC']),:])}")
df4 = df3.loc[~df["IVA_COMPANY_RATING"].isin(['B','CCC']),:]
print(f"Elimination {len(df3)-len(df4)}")
print(f"Reste {len(df4)}")

NB : 17
Elimination 11
Reste 1051


### Restrictions ODD

In [83]:
# odds = ["07","12","13"]
odds = [x for x in df.columns if x.startswith("SDG")]
for odd in odds:
    print(f"NB {odd} : {len(df.loc[df[odd]<=-2,:])}")

df5 = df4.loc[~df4[odds].le(-2).any(axis=1), :]
print(f"Elimination {len(df4)-len(df5)}")
print(f"Reste {len(df5)}")

NB SDG_01_NET_ALIGNMENT_SCORE : 28
NB SDG_02_NET_ALIGNMENT_SCORE : 7
NB SDG_03_NET_ALIGNMENT_SCORE : 48
NB SDG_04_NET_ALIGNMENT_SCORE : 16
NB SDG_05_NET_ALIGNMENT_SCORE : 1
NB SDG_06_NET_ALIGNMENT_SCORE : 14
NB SDG_07_NET_ALIGNMENT_SCORE : 83
NB SDG_08_NET_ALIGNMENT_SCORE : 6
NB SDG_09_NET_ALIGNMENT_SCORE : 37
NB SDG_10_NET_ALIGNMENT_SCORE : 9
NB SDG_11_NET_ALIGNMENT_SCORE : 51
NB SDG_12_NET_ALIGNMENT_SCORE : 106
NB SDG_13_NET_ALIGNMENT_SCORE : 83
NB SDG_14_NET_ALIGNMENT_SCORE : 64
NB SDG_15_NET_ALIGNMENT_SCORE : 66
NB SDG_16_NET_ALIGNMENT_SCORE : 28
NB SDG_17_NET_ALIGNMENT_SCORE : 70
Elimination 106
Reste 945


In [117]:
(df5['Poids_Optimisé']*df5['Annualized return 20Y']).sum()

0.4214695064369942

In [99]:
df5['EST_EU_TAXONOMY_MAX_REV'] = df5['EST_EU_TAXONOMY_MAX_REV'].fillna(0)
df5['Annualized return 10Y'] = df5['Annualized return 10Y'].fillna(0)

C:\Users\bapdu\AppData\Local\Temp\ipykernel_7064\2480137706.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['EST_EU_TAXONOMY_MAX_REV'] = df5['EST_EU_TAXONOMY_MAX_REV'].fillna(0)
C:\Users\bapdu\AppData\Local\Temp\ipykernel_7064\2480137706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Annualized return 10Y'] = df5['Annualized return 10Y'].fillna(0)


In [118]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize


# Nombre de tickers
n = len(df5)

perf_moy = df5['Annualized return 10Y'].values
taxonomie = df5['EST_EU_TAXONOMY_MAX_REV'].values

# Fonction objectif (on minimise donc on prend -Perf_moy)
def objectif(w):
    return -np.dot(w, perf_moy)  # On minimise la valeur négative pour maximiser

# Contrainte : Taxonomie pondérée ≥ 10
def contrainte_taxonomie(w):
    return np.dot(w, taxonomie)-10

# # Contrainte : Alignement environnemental pondéré ≥ 90%
# def contrainte_alignement(w):
#     return np.dot(w, alignement_env) - 0.90

# Contrainte : Somme des poids = 1
def contrainte_somme(w):
    return np.sum(w) - 1

# Définition des contraintes sous forme de dictionnaires
contraintes = [
    {'type': 'eq', 'fun': contrainte_somme},  # Somme des poids = 1
    {'type': 'ineq', 'fun': contrainte_taxonomie},  # Taxonomie pondérée ≥ 10
    # {'type': 'ineq', 'fun': contrainte_alignement}  # Alignement pondéré ≥ 90%
]

# Bornes (chaque poids doit être positif et max 0.1)
bornes = [(0, 0.05) for _ in range(n)]

# Initialisation des poids (égalité répartie)
w0 = np.ones(n) / n

# Résolution avec SLSQP
resultat = minimize(objectif, w0, method='trust-constr', bounds=bornes, constraints=contraintes)

# Vérification des résultats
if resultat.success:
    w_opt = resultat.x
    df5['Poids_Optimisé'] = w_opt
else:
    print("L'optimisation a échoué :", resultat.message)


c:\Users\bapdu\anaconda3\Lib\site-packages\scipy\optimize\_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


KeyboardInterrupt: 

In [115]:
df5[df5['Poids_Optimisé']>=0.04]

,ticker,name,isin,country,currency,gics_sector_name,industry_group,region_en,Weight in MSCI World,Annualized return 20Y,...,SCOPE_3,FOOTPRINT_SCOPE_1,FOOTPRINT_SCOPE_2,FOOTPRINT_SCOPE_3,INTENSITY_SCOPE_1,INTENSITY_SCOPE_2,INTENSITY_SCOPE_3,EV,SALES,Poids_Optimisé
1,NVDA US Equity,NVIDIA Corp,US67066G1040,UNITED STATES,USD,Information Technology,Semiconductors,North America,0.046132,0.396705,...,27507573.61,0.018164,0.224788,34.721009,0.131528,1.627755,251.425343,3.359642e+06,109406.527100,0.05000
7,AVGO US Equity,Broadcom Inc,US11135F1012,UNITED STATES,USD,Information Technology,Semiconductors,North America,0.015473,0.317303,...,13777289.32,0.297687,0.942548,69.793000,1.179637,3.735015,276.567279,8.371146e+05,49815.326600,0.05000
19,NFLX US Equity,Netflix Inc,US64110L1061,UNITED STATES,USD,Communication Services,Internet,North America,0.005515,0.376162,...,8447086.96,0.353979,0.415922,115.939906,0.711690,0.836228,233.102162,3.089634e+05,36237.703140,0.05000
33,NOW US Equity,ServiceNow Inc,US81762P1021,UNITED STATES,USD,Information Technology,Software,North America,0.003177,0.232904,...,979094.31,0.058149,1.315634,23.429482,0.240423,5.439600,96.871189,1.772127e+05,10107.177600,0.05000
47,AMD US Equity,Advanced Micro Devices Inc,US0079031078,UNITED STATES,USD,Information Technology,Semiconductors,North America,0.002841,0.101752,...,10292150.34,0.164830,1.331401,169.509509,0.426480,3.444862,438.588310,2.574809e+05,23466.540500,0.05000
91,SHOP CN Equity,Shopify Inc,CA82509L1076,CANADA,CAD,Information Technology,Internet,North America,0.001814,0.205190,...,22360.00,0.021991,0.054832,0.722056,0.063080,0.157284,2.071180,1.313209e+05,10795.776000,0.05000
108,ANET US Equity,Arista Networks Inc,US0404132054,UNITED STATES,USD,Information Technology,Telecommunications,North America,0.001584,0.216895,...,2329704.69,0.086769,0.534863,88.505781,0.357566,2.204101,364.720800,1.116252e+05,6387.638687,0.05000
270,MSTR US Equity,MicroStrategy Inc,US5949724083,UNITED STATES,USD,Information Technology,Software,North America,0.000772,0.225793,...,57196.58,0.029618,0.342928,6.699589,0.560280,6.487093,126.734569,3.620386e+04,451.310014,0.05000
272,TTD US Equity,Trade Desk Inc,US88339J1051,UNITED STATES,USD,Communication Services,Advertising,North America,0.000766,0.232504,...,380145.59,0.030514,0.439957,31.098500,0.167200,2.410732,170.403304,5.183744e+04,2230.858094,0.05000
295,FICO US Equity,Fair Isaac Corp,US3032501047,UNITED STATES,USD,Information Technology,Software,North America,0.000693,0.234323,...,244245.57,0.114720,1.148686,21.090629,0.800828,8.018682,147.228270,4.911003e+04,1658.958363,0.05000
